In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
!pip install BeautifulSoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

In [ ]:
table1 = soup.find('table',{'class':'wikitable sortable'})
table1
#This also works:
#table = soup.table.text
#print(table)

In [ ]:
print(table1.tr.text)

In [ ]:
header=["PostalCode","Borough","Neighborhood"]

df = pd.DataFrame(columns=header)
df

In [ ]:
table1_rows = table1.find_all('tr')

for tr in table1_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)
    if row!=[]: #first row is empty. This could be done better
        postcode = row[0]
        borough = row[1]
        neighborhood = row[2]
    
        df = df.append({'PostalCode': postcode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood}, ignore_index=True)

In [ ]:
df.head()

We need to clean this up now. We will delete the rows where the borough is not assigned.

In [ ]:
df.drop(df[df.Borough.str.contains('Not assigned')].index, inplace = True)

In [ ]:
df

In [ ]:
df.reset_index(inplace=True,drop=True)
df

Now let's remove the new line tags: "\n"

In [ ]:
df = df.replace('\n','', regex=True)

In [ ]:
df

In [ ]:
df['Neighborhood'] = df['Neighborhood'].replace(' /',',', regex=True)

In [ ]:
df

In [ ]:
df.shape

QUESTION 2
The assignment notes that the Geocoder package is unreliable at times, so the csv file with postal codes is provided in the link below.

In [ ]:

df_codes = pd.read_csv('https://cocl.us/Geospatial_data')
df_codes.head()

In [ ]:
df_codes.rename(columns={"Postal Code": "PostalCode"}, inplace = True) #We will rename the first column to match our original dataframe

In [ ]:

df1 = pd.merge(df, df_codes[['PostalCode','Latitude','Longitude']], on='PostalCode') #Let's now merge both data frames

In [ ]:

df1 #final data frame for question 2


Question 3
In this section we'll explore and cluster the neighborhoods in Toronto.
First we use Folium to map Toronto.

In [ ]:
# create map of Toronto using latitude and longitude values
latitude = 43.651070
longitude = -79.347015
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Enter your API credentials to access the Foursquare API.

In [ ]:
CLIENT_ID = 'yourid' # your Foursquare ID
CLIENT_SECRET = 'yoursecret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

The following function will make requests to the API for all the neighborhoods in our dataframe.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
So, let's run the function.

In [ ]:
tor_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )

In [ ]:
print(tor_venues.shape)
tor_venues.head()

We have the dataframe above with the different Venues (last column!) for each Neighborhood. How many venues we have for each neighborhood?

In [ ]:
tor_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

Here we start our analysis of each neighborhood and prepare to cluster them.
First step is to turn the venue categories into dummy variables.

In [ ]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()


We can group the df by neighborhood and find the mean occurrence of venues per neighborhood

In [ ]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

In [ ]:
tor_grouped.shape

Let's check the top 5 venues for each neighborhood.

In [ ]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

This function will sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we create a new dataframe with the 10 most common venues per neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

We can cluster these neighborhoods based on the similarity of the venues in each of them.
We are applying k-means here, with 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

Let's merge the previous dataframe with the new cluster labels.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

tor_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

There are some NAs in the dataframe, we will drop these. Plus the clusters are in float. Let's change to integer.

In [ ]:
tor_merged=tor_merged.dropna()
tor_merged['Cluster_Labels'] = tor_merged.Cluster_Labels.astype(int)

Let's map Toronto once again, but with the clusters represented by different colors.

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

With the following code, we can explore a specific cluster.

In [ ]:
tor_merged.loc[tor_merged['Cluster_Labels'] == 0, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]